In [1]:
import tensorflow as tf
source = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'
model_dir = tf.keras.utils.get_file(fname='ssd_mobilenet_v2_320x320_coco17_tpu-8',origin = source, untar=True)

2023-03-10 13:49:34.313825: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 13:49:34.314038: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = tf.saved_model.load(model_dir+'/saved_model',tags = ['serve'])

2023-03-10 13:50:43.097280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-10 13:50:43.100885: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-10 13:50:43.102380: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Magnus): /proc/driver/nvidia/version does not exist
2023-03-10 13:50:43.119644: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
signature = model.signatures['serving_default']

In [4]:
import numpy as np
import cv2
# Load an image from a file
image = cv2.imread('Screenshot 2023-03-10 121618.jpg')

# Convert the image from BGR to RGB format
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Resize the image to 320x320 pixels
image = cv2.resize(image, (320, 320))

# Convert the image to a float32 tensor
image = tf.convert_to_tensor(image, dtype=tf.uint8)

# Add a batch dimension to the tensor
image = tf.expand_dims(image, axis=0)

output_tensor = signature(image)


2023-03-10 13:51:49.301841: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 8640000 exceeds 10% of free system memory.
